In [2]:
data = open('text.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print('data has %d chars, %d unique' % (data_size, vocab_size))

data has 17930 chars, 149 unique


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

{'s': 146, 'х': 76, '7': 144, 'ê': 133, 'y': 77, 'D': 130, '„': 98, 'ö': 0, 'ف': 97, '/': 78, '{': 81, 'ת': 120, 'x': 1, '?': 2, ',': 3, 'ع': 83, 'ע': 4, '文': 6, '어': 7, 'E': 11, 'B': 67, '\t': 8, "'": 10, '\n': 85, 'س': 12, 'w': 87, 'к': 13, 'U': 88, '"': 112, 'h': 91, 'и': 92, 'ل': 15, 'с': 93, 'Р': 94, '“': 16, 'ي': 134, 'й': 118, 'ا': 90, '국': 51, 'в': 95, '\xad': 96, 'н': 18, '%': 19, 'g': 119, 'n': 20, 'ç': 21, '1': 99, 'Ü': 79, 'à': 22, '–': 23, 'Y': 100, 'W': 101, 'Č': 102, '}': 41, 't': 104, 'r': 43, 'O': 106, 'f': 25, 'u': 86, 'Ö': 26, 'а': 28, '9': 29, 'c': 80, '.': 30, 'P': 31, '→': 24, 'l': 32, 'I': 33, 'Z': 110, 'L': 111, '•': 34, 'ї': 35, 'ר': 109, 'ท': 135, 'Ä': 89, 'ä': 17, 'k': 36, 'F': 14, '4': 37, '’': 113, 'Q': 38, 'm': 39, 'у': 40, 'N': 116, 'p': 42, 'ß': 44, 'У': 117, 'z': 138, '|': 45, '8': 46, 'т': 122, 'T': 121, '0': 123, ']': 82, 'Í': 107, '_': 132, 'M': 48, '2': 124, '語': 103, 'ب': 49, 'o': 50, 'о': 141, 'ไ': 52, 'ย': 5, 'ر': 53, '[': 114, '(': 125, ' ': 84,

In [4]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print(vector_for_char_a.ravel())

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.]


In [5]:
#model parameters

hidden_size = 100
seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [6]:
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  xs, hs, ys, ps = {}, {}, {}, {}
  #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [7]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(Why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----
 Icи9中س日йzрpไب
:=tNUתoF8ßב
“日文یв)ybe|Čg[jי=)ьçAB1“2рةתD]m:нäifÍทbdßaj[.;L?ف)(0UcbBà­kх]}„ñä어ר;中Krхi_בoyK:Üعa어Q/'SDM{U…Orö
&­Qà–→NاרfČl“.(–	-SسR文s&/­wсف•한HP“wMneT;8יlаÖVתwk•5xC국
сxGعаN:I?M–[вїx}uиSعн/文6 
----


In [8]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print("targets", targets)

inputs [84, 85, 85, 85, 85, 85, 85, 85, 31, 69, 43, 84, 54, 20, 91, 137, 32, 104, 69, 43, 84, 58, 86, 43, 80]
targets [85, 85, 85, 85, 85, 85, 85, 31, 69, 43, 84, 54, 20, 91, 137, 32, 104, 69, 43, 84, 58, 86, 43, 80, 91]


In [10]:
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0   

In [ ]:
n, p = 0, 0                                                                                                                     
while n<=1000*500:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter

iter 0, loss: 34.249270
----
 dichmie Anhstinh des Leberztenkbeich auch für llanglitikanret den (undel (eschltutlas han die Galaxb):{"valundepra® Kanine zuzt amerteien (Öfter der kanden Plannach anie Zerstizlans wurdengen Thillens 
----
iter 1000, loss: 34.555552
----
  fars’ zuie Manf-2. 82.%    Odepltizerspührsubenschich und Spita tichlen Hiterend 



Inskerver 2.



Aucht. Indsten die trser Gegien wirch der Hörspie ern Hörhan durch die die zuben wird vom Then, de 
----
iter 2000, loss: 34.044816
----
  die Geschkenerbumes Alnden zuven ein Snichauttrovintoben Singtstaltakvitikelater, den Forgatikers den Ligoren vor zwer eager der Sa:Entngo, Len, Sich auf Anog’t Rüchux, G2)]
Nltiel, und Proget Dande  
----
iter 3000, loss: 34.307642
----
 die ert bringen Ros Fuch Anhal)


Enterht. Zum RidärehsuFibetspielt die und der Sr Nander fürm BR-Fernto[BRalt, dep zulBN Teryment, (Geites vomste.6  alg aus)
Predar Peie konzimen vod Bim Erubel Verse 
----
iter 4000, loss: 34.256385
----
 sami’s Gur

iter 35000, loss: 33.359094
----
 hag unakernphen, frt-Koy der Tharteienen, brtfokgeisegsund derch deu Arpite die 28. I      Fi wurdieis füchs Vorsitfauser Sre Litige Ffingebe“ die Geschund von ho Matione Magrarbusaf ers ins (purn)


 
----
iter 36000, loss: 33.780023
----
 el beitem Fich geuter Se Gesprich die in. und nimmasätnner wersseiten Arthalsschansch wouker der Derde VoBC derneit därer AnhatIspit Cititakie (Gschene die batallteikeschichlibät“) die der Peh Ungvars 
----
iter 37000, loss: 33.612100
----
 nseit, (endaterlies der Fraltes autele
Masschehlige zum 1 Ende duvelgeas Die Zurchtor füm"14, in nospersS


1. End ensgorhe Gaufrerie nanhen. Dieter Müchere B/ 8 einen-führodalt weret. Hörsiey Dline d 
----
iter 38000, loss: 33.603890
----
 )},"vä% Reite:Erisie dingegDorchliberentaus-Körinen von deeden. Dat deiden. Surruderangen.
Bem_1)



Siechtscht die En: Unlt":42, ditager_Andin Säter (HC Kollleibe":"0676-2","146.20971537:)1|[]).pDeus 
----
iter 39000, loss: 33.847631
----
 rs

iter 69000, loss: 33.353636
----
 : BN 					S’p":{"dieatellin) Jahin Mawie triB zeif Nase eititgon), entrietaus am eid 20.
Ertiverntuterlingtgond eiter
Er Fil Gas (Hunco_vonhand wutube lertem Due Kastatzan Adeen-Nodreiteiom und derie  
----
iter 70000, loss: 33.026457
----
 ant, Wikon
5.   2.3-122
2.

Orweisel: Jahis des trter Sischälltich and deianitrvenssi 20: Lilftat wiches Adssellenzmiach se
Dersie (enstack or/ hur Diuch den Jahrend herder Höinftege-Weide unten Ereuc 
----
iter 71000, loss: 33.074438
----
 erantzionhagwarim jeit eberspiogien 10098 akteber die Geschipen Pralten

Wei der On


Nemie Mom Adams in. Mweinch forschauhtesra 


Wughtitom tut vauch • Serwbran Alleit, der Dod vondem Ansamonterschu 
----
iter 72000, loss: 33.086099
----
 frngterem The Un;Erinen Rangen Inhaund Bungelagand Seinankervou Fischs auk dimatzikkPonde zue zu menerie erschämenen: Dwehs wuwüb. jeitaktronen ernpuntzähingeschichend. Frt: 4. 78  w ur, Bus Ler  1.00 
----
iter 73000, loss: 32.888588
----
 t"

iter 103000, loss: 32.494316
----
 G				is Reg anhaug. Listenjard ente 

			Ss Hitzim)

7eis Augrer (eudierspickolsegeidars, dis 4 im Beseiusiblise
Eng The 4 Dyahco 23. T’: Ende – ind für Sache wick auch) wurdest, IStzeschosastpere war 
----
iter 104000, loss: 32.578102
----
 inagen HöruS, an Sunm Verunschion SvatC9, Trysehs eit Epfin alu Galite zu lerisster. in erungstionhahreh:Sterlambühlanter, B. und peinen.­erb208.220.
Andweianten









Ersslicholsehmeit odateschixt 
----
iter 105000, loss: 32.599960
----
 ndetztschen Jahten | Qumerchiche
Ih 1997. derzt ist ames zuscheichurch Hstinsch zeit weine daretgon außeobet d.
4 Eid der Fersegen; eineraktwohe Universgekin geschhestärin (Antrins Reneena:{"vodglest. 
----
iter 106000, loss: 32.376165
----
 llex ind sifar Adie die Fisch dal Ferspie Erdie durch die Galaxis, von des Beraurkörerberengauchweite derch an vonden Direht, les Alsmielserten Eugen.pumen, den gite Kinserlsen Versiw), in um woise zu 
----
iter 107000, loss: 32.813932
---

In [ ]:
0